In [1]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig, get_linear_schedule_with_warmup, DataCollatorWithPadding
from torch.utils.data import Dataset, DataLoader
import torch
from tqdm.notebook import tqdm
import random
import os

In [2]:
BATCH_SIZE = 6

In [3]:
def seed_all(seed):
    random.seed(seed)
    os.environ['PYTHONDASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
seed_all(42)

In [4]:
class NewsDataset(Dataset):
    def __init__(self, data, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.code = data['code'].to_numpy()
        self.problem = data['label'].to_numpy()

    def __len__(self):
        return len(self.code)

    def __getitem__(self, idx):
        code1 = self.code[idx]
        standard = 500*self.problem[idx]
        r = np.random.random()
        #good
        if r < 0.5:
            tmp = np.random.randint(standard, standard + 500)
            code2 = self.code[tmp]
            label = 1
        #bad
        else:
            tmp = np.random.randint(standard + 500, len(self.code) + standard) % len(self.code)
            code2 = self.code[tmp]
            label = 0
        encoding = self.tokenizer(
            code1,
            code2,
            #add_special_tokens=True,
            max_length=self.max_len,
            truncation=True,
            return_tensors="pt",
            padding='max_length'
        )

        return {'input_ids': encoding['input_ids'][0],
                'attention_mask': encoding['attention_mask'][0],
                #'token_type_ids' : encoding['token_type_ids'][0],
                'labels': torch.tensor(label, dtype=torch.long)}

In [5]:
#model_name = "neulab/codebert-cpp"
#model_name = 'neulab/codebert-cpp'
#model_name = 'microsoft/graphcodebert-base'

model_name = 'codesage/codesage-small'
train_data = pd.read_csv('./siba.csv')


tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.truncation_side = 'left'

config = AutoConfig.from_pretrained(model_name, trust_remote_code=True)
train_dataset = NewsDataset(train_data, tokenizer, max_len=1024)

train_loader = DataLoader(train_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True,
                          pin_memory=True,
                          num_workers=4)

config.problem_type = "single_label_classification"
config.num_labels = 2
config.classifier_dropout = None
model = AutoModelForSequenceClassification.from_pretrained(
        model_name, config=config, trust_remote_code=True
    )
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
#scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps=1,num_training_steps=5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1, last_epoch=-1)

config.json:   0%|          | 0.00/792 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/512M [00:00<?, ?B/s]

Some weights of CodeSageForSequenceClassification were not initialized from the model checkpoint at codesage/codesage-small and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
model.zero_grad()
model.train()
for epoch in tqdm(range(0,6,1)):
    model.train()
    train_loss = 0
    acc = 0
    tmp_acc = 0
    cnt = 0
    print(f'\nepoch : {epoch+1}')
    for i, batch in tqdm(enumerate(train_loader),leave=False,total=len(train_loader)):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        #token_type_ids = batch['token_type_ids'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        logits = outputs.logits
        preds = torch.argmax(logits, axis=1)
        acc += torch.sum(labels==preds)
        tmp_acc += torch.sum(labels==preds)
        cnt += torch.sum(preds)
        loss = outputs.loss
        train_loss += loss.item()
        loss.backward()
        if i%500==499:
            print(tmp_acc.item()/ (500*BATCH_SIZE))
            model.save_pretrained(f"codesage-final-/cpp-{epoch}", from_pt=True)
            tmp_acc = 0
        optimizer.step()
        model.zero_grad()
    scheduler.step()
    print(f'train_loss : {train_loss}\nacc = {acc / len(train_data)}\ncount = {cnt}')

  0%|          | 0/6 [00:00<?, ?it/s]


epoch : 1


  0%|          | 0/41667 [00:00<?, ?it/s]

0.5483333333333333
0.771
0.875
0.8943333333333333
0.9123333333333333
0.912
0.922
0.93
0.9253333333333333
0.9303333333333333
0.937
0.9456666666666667
0.94
0.9376666666666666
0.9456666666666667
0.9426666666666667
0.952
0.9573333333333334
0.949
0.948
0.9513333333333334
0.956
0.953
0.951
0.9503333333333334
0.9543333333333334
0.959
0.9576666666666667
0.958
0.9553333333333334
0.9613333333333334
0.9533333333333334
0.9596666666666667
0.9583333333333334
0.954
0.9646666666666667
0.9643333333333334
0.9656666666666667
0.9676666666666667
0.9616666666666667
0.969
0.964
0.9653333333333334
0.9633333333333334
0.9683333333333334
0.965
0.96
0.9646666666666667
0.9673333333333334
0.9756666666666667
0.9623333333333334
0.9706666666666667
0.9696666666666667
0.9653333333333334
0.971
0.968
0.964
0.9676666666666667
0.9723333333333334
0.9676666666666667
0.9696666666666667
0.972
0.965
0.969
0.9656666666666667
0.9686666666666667
0.9716666666666667
0.9663333333333334
0.971
0.966
0.9706666666666667
0.969
0.9743333333

  0%|          | 0/41667 [00:00<?, ?it/s]

...

0.9866666666666667
0.9913333333333333
0.9876666666666667
0.9883333333333333
0.9896666666666667
0.989
0.989
0.988
0.9876666666666667
0.9896666666666667
0.9916666666666667
0.994
0.9896666666666667
0.9886666666666667
0.988
0.986
0.9933333333333333
0.9903333333333333
0.9893333333333333
0.9903333333333333
0.989
0.9896666666666667
0.9883333333333333
0.992
0.9923333333333333
0.9906666666666667
0.9906666666666667
0.9873333333333333
0.99
0.9916666666666667
0.9896666666666667
0.9926666666666667
0.9873333333333333
0.9886666666666667
0.989
0.991
0.9886666666666667
0.9896666666666667
0.989
0.993
0.9906666666666667
0.9886666666666667
0.9893333333333333
0.9913333333333333
0.9893333333333333
0.9893333333333333
0.988
0.9876666666666667
0.9876666666666667
0.991
0.9893333333333333
0.9916666666666667
0.992
0.9903333333333333
0.9903333333333333
0.9896666666666667
0.9886666666666667
0.9893333333333333
0.9846666666666667
0.9873333333333333
0.9903333333333333
0.9853333333333333
0.991
0.9923333333333333
0.9873

  0%|          | 0/41667 [00:00<?, ?it/s]

0.992
0.9896666666666667
0.9906666666666667
0.9926666666666667
0.9883333333333333
0.9886666666666667
0.9893333333333333
0.988
0.9933333333333333
0.9916666666666667
0.991
0.9873333333333333
0.9936666666666667
0.9916666666666667
0.9916666666666667
0.9906666666666667
0.988
0.988
0.9933333333333333
0.9906666666666667
0.991
0.991
0.991
0.9893333333333333
0.989
0.9906666666666667
0.991
0.9906666666666667
0.9893333333333333
0.9913333333333333
0.9886666666666667
0.991
0.9956666666666667
0.9916666666666667
0.9906666666666667
0.9906666666666667
0.9923333333333333
0.9946666666666667
0.9933333333333333
0.994
0.993
0.9903333333333333
0.9913333333333333
0.993
0.9923333333333333
0.9893333333333333
0.9903333333333333
0.9946666666666667
0.9916666666666667
0.9893333333333333
0.9926666666666667
0.9846666666666667
0.991
0.9936666666666667
0.991
0.992
0.9913333333333333
0.9916666666666667
0.99
0.99
0.9946666666666667
0.991
0.9896666666666667
0.9913333333333333
0.991
0.991
0.9906666666666667
0.992
0.9933333

  0%|          | 0/41667 [00:00<?, ?it/s]

0.9936666666666667
0.9923333333333333
0.9866666666666667
0.993
0.9926666666666667
0.9873333333333333
0.994
0.9926666666666667
0.9913333333333333
0.9916666666666667
0.9916666666666667
0.9953333333333333
0.9903333333333333
0.9926666666666667
0.9933333333333333
0.989
0.9936666666666667
0.9873333333333333
0.9906666666666667
0.992
0.9906666666666667
0.9923333333333333
0.9883333333333333
0.9896666666666667
0.991
0.9903333333333333
0.9906666666666667
0.9916666666666667
0.9886666666666667
0.989
0.992
0.991
0.991
0.9936666666666667
0.9923333333333333
0.992
0.9903333333333333
0.9933333333333333
0.99
0.9903333333333333
0.9943333333333333
0.9916666666666667
0.9903333333333333
0.9946666666666667
0.99
0.9936666666666667
0.9876666666666667
0.992
0.993
0.9903333333333333
0.9903333333333333
0.9906666666666667
0.9906666666666667
0.9883333333333333
0.99
0.9883333333333333
0.993
0.9913333333333333
0.9916666666666667
0.9916666666666667
0.9926666666666667
0.9923333333333333
0.9906666666666667
0.989666666666

In [7]:
torch.save(optimizer,'./sage_opt.pt')

In [7]:
model.zero_grad()
model.train()
for epoch in tqdm(range(0,6,1)):
    model.train()
    train_loss = 0
    acc = 0
    tmp_acc = 0
    cnt = 0
    print(f'\nepoch : {epoch+1}')
    for i, batch in tqdm(enumerate(train_loader),leave=False,total=len(train_loader)):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        #token_type_ids = batch['token_type_ids'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        logits = outputs.logits
        preds = torch.argmax(logits, axis=1)
        acc += torch.sum(labels==preds)
        tmp_acc += torch.sum(labels==preds)
        cnt += torch.sum(preds)
        loss = outputs.loss
        train_loss += loss.item()
        loss.backward()
        if i%500==499:
            print(tmp_acc.item()/ (500*BATCH_SIZE))
            model.save_pretrained(f"codesage-final-/cpp-{epoch}", from_pt=True)
            tmp_acc = 0
        optimizer.step()
        model.zero_grad()
    scheduler.step()
    print(f'train_loss : {train_loss}\nacc = {acc / len(train_data)}\ncount = {cnt}')

  0%|          | 0/5 [00:00<?, ?it/s]


epoch : 1


  0%|          | 0/41667 [00:00<?, ?it/s]

0.5226666666666666
0.512
0.594
0.782
0.8723333333333333
0.891
0.914
0.9226666666666666
0.926
0.928
0.9353333333333333
0.941
0.9376666666666666
0.9386666666666666
0.9453333333333334
0.9436666666666667
0.9533333333333334
0.9493333333333334
0.9453333333333334
0.9466666666666667
0.951
0.947
0.9553333333333334
0.9536666666666667
0.9456666666666667
0.9526666666666667
0.9576666666666667
0.9576666666666667
0.9523333333333334
0.958
0.959
0.9556666666666667
0.9576666666666667
0.9493333333333334
0.96
0.9643333333333334
0.9626666666666667
0.9656666666666667
0.962
0.9613333333333334
0.9703333333333334
0.9636666666666667
0.963
0.9683333333333334
0.9683333333333334
0.9696666666666667
0.9596666666666667
0.966
0.9663333333333334
0.9676666666666667
0.965
0.9666666666666667
0.963
0.9673333333333334
0.9703333333333334
0.9706666666666667
0.9706666666666667
0.9666666666666667
0.9753333333333334
0.9673333333333334
0.9716666666666667
0.9673333333333334
0.965
0.971
0.97
0.967
0.9653333333333334
0.975
0.975
0.9

  0%|          | 0/41667 [00:00<?, ?it/s]

...

0.9846666666666667
0.9846666666666667
0.9823333333333333
0.9806666666666667
0.9856666666666667
0.988
0.9843333333333333
0.9863333333333333
0.983
0.987
0.98
0.9866666666666667
0.9873333333333333
0.9866666666666667
0.983
0.9873333333333333
0.9836666666666667
0.9833333333333333
0.9893333333333333
0.9896666666666667
0.9863333333333333
0.986
0.9883333333333333
0.983
0.9813333333333333
0.986
0.9856666666666667
0.9846666666666667
0.9856666666666667
0.9893333333333333
0.9893333333333333
0.9856666666666667
0.989
0.9893333333333333
0.9853333333333333
0.986
0.987
0.9866666666666667
0.99
0.9863333333333333
0.9893333333333333
0.988
0.9873333333333333
0.9853333333333333
0.9876666666666667
0.99
0.9886666666666667
0.988
0.9886666666666667
0.9836666666666667
0.989
0.9883333333333333
0.9913333333333333
0.99
0.9873333333333333
0.989
0.9873333333333333
0.9873333333333333
0.988
0.9916666666666667
0.987
0.988
0.9893333333333333
0.992
0.988
0.9886666666666667
0.9903333333333333
0.9916666666666667
0.988333333

  0%|          | 0/41667 [00:00<?, ?it/s]

0.9873333333333333
0.99
0.987
0.9896666666666667
0.9893333333333333
0.992
0.9913333333333333
0.9876666666666667
0.988
0.9893333333333333
0.9923333333333333
0.9936666666666667
0.9873333333333333
0.9886666666666667
0.989
0.9876666666666667
0.9916666666666667
0.9903333333333333
0.987
0.992
0.9896666666666667
0.993
0.9893333333333333
0.9906666666666667
0.992
0.989
0.9883333333333333
0.991
0.9896666666666667
0.9916666666666667
0.989
0.99
0.989
0.9876666666666667
0.985
0.9913333333333333
0.9873333333333333
0.989
0.9883333333333333
0.991
0.9886666666666667
0.9843333333333333
0.9876666666666667
0.992
0.9896666666666667
0.9893333333333333
0.991
0.986
0.9883333333333333
0.989
0.9906666666666667
0.989
0.994
0.9896666666666667
0.9906666666666667
0.991
0.985
0.9906666666666667
0.9896666666666667
0.99
0.9866666666666667
0.9873333333333333
0.991
0.9896666666666667
0.989
0.988
0.9913333333333333
0.9913333333333333
0.9886666666666667
0.9903333333333333
0.9913333333333333
0.9863333333333333
0.9923333333

  0%|          | 0/41667 [00:00<?, ?it/s]

0.992
0.9873333333333333
0.993
0.9906666666666667
0.9876666666666667
0.9913333333333333
0.9913333333333333
0.9856666666666667
0.9933333333333333
0.9923333333333333
0.9883333333333333
0.99
0.993
0.994
0.9916666666666667
0.9913333333333333
0.989
0.9913333333333333
0.9896666666666667
0.991
0.9893333333333333
0.9916666666666667
0.9906666666666667
0.9936666666666667
0.9896666666666667
0.992
0.9906666666666667
0.9896666666666667
0.9896666666666667
0.992
0.9903333333333333
0.9893333333333333
0.9936666666666667
0.9893333333333333
0.9906666666666667
0.9913333333333333
0.992
0.992
0.9926666666666667
0.9943333333333333
0.9916666666666667
0.991
0.9906666666666667
0.9926666666666667
0.9903333333333333
0.9906666666666667
0.9906666666666667
0.9933333333333333
0.9923333333333333
0.9913333333333333
0.991
0.9833333333333333
0.993
0.993
0.9906666666666667
0.9913333333333333
0.9923333333333333
0.989
0.9893333333333333
0.989
0.9933333333333333
0.9913333333333333
0.9906666666666667
0.9903333333333333
0.9923

In [8]:
for name, param in model.named_parameters():
    if name.startswith("transformer"): # choose whatever you like here
        param.requires_grad = False
    if name.startswith('transformer.h.5'):
        param.requires_grad = True
    if name.startswith('transformer.ln_f'):
        param.requires_grad = True

for name, param in model.named_parameters():
    print(name, param.requires_grad)
BATCH_SIZE = 32

transformer.wte.weight False
transformer.wpe.weight False
transformer.h.0.ln_1.weight False
transformer.h.0.ln_1.bias False
transformer.h.0.attn.c_attn.weight False
transformer.h.0.attn.c_attn.bias False
transformer.h.0.attn.c_proj.weight False
transformer.h.0.attn.c_proj.bias False
transformer.h.0.ln_2.weight False
transformer.h.0.ln_2.bias False
transformer.h.0.mlp.c_fc.weight False
transformer.h.0.mlp.c_fc.bias False
transformer.h.0.mlp.c_proj.weight False
transformer.h.0.mlp.c_proj.bias False
transformer.h.1.ln_1.weight False
transformer.h.1.ln_1.bias False
transformer.h.1.attn.c_attn.weight False
transformer.h.1.attn.c_attn.bias False
transformer.h.1.attn.c_proj.weight False
transformer.h.1.attn.c_proj.bias False
transformer.h.1.ln_2.weight False
transformer.h.1.ln_2.bias False
transformer.h.1.mlp.c_fc.weight False
transformer.h.1.mlp.c_fc.bias False
transformer.h.1.mlp.c_proj.weight False
transformer.h.1.mlp.c_proj.bias False
transformer.h.2.ln_1.weight False
transformer.h.2.ln_1

In [9]:
train_loader = DataLoader(train_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True,
                          pin_memory=True,
                          num_workers=4)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps=1,num_training_steps=5)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1, last_epoch=-1)

In [10]:
model.zero_grad()
model.train()
for epoch in tqdm(range(0,4,1)):
    model.train()
    train_loss = 0
    acc = 0
    tmp_acc = 0
    cnt = 0
    print(f'\nepoch : {epoch+1}')
    for i, batch in tqdm(enumerate(train_loader),leave=False,total=len(train_loader)):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        #token_type_ids = batch['token_type_ids'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        logits = outputs.logits
        preds = torch.argmax(logits, axis=1)
        acc += torch.sum(labels==preds)
        tmp_acc += torch.sum(labels==preds)
        cnt += torch.sum(preds)
        loss = outputs.loss
        train_loss += loss.item()
        loss.backward()
        if i%100==99:
            print(tmp_acc.item()/ (100*BATCH_SIZE))
            model.save_pretrained(f"codesage-small-left-test-freeze/cpp-{epoch}", from_pt=True)
            tmp_acc = 0
        optimizer.step()
        model.zero_grad()
    scheduler.step()
    print(f'train_loss : {train_loss}\nacc = {acc / len(train_data)}\ncount = {cnt}')

  0%|          | 0/5 [00:00<?, ?it/s]


epoch : 1


  0%|          | 0/7813 [00:00<?, ?it/s]

0.9896875
0.9915625
0.990625
0.9915625
0.9915625
0.993125
0.9890625
0.9915625
0.990625
0.9890625
0.9915625
0.9896875
0.9878125
0.9890625
0.9909375
0.9909375
0.9896875
0.99
0.9921875
0.991875
0.993125
0.9865625
0.9925
0.9940625
0.9934375
0.990625
0.99
0.9925
0.991875
0.9921875
0.98875
0.990625
0.9909375
0.9925
0.98875
0.9934375
0.99
0.990625
0.99
0.995625
0.9909375
0.993125
0.9903125
0.9925
0.993125
0.991875
0.9903125
0.99125
0.9875
0.9925
0.995
0.9909375
0.9915625
0.99
0.99
0.9903125
0.99125
0.99125
0.9953125
0.9915625
0.9884375
0.9903125
0.9884375
0.99125
0.9915625
0.9928125
0.98875
0.9890625
0.993125
0.99375
0.9915625
0.989375
0.9921875
0.991875
0.9890625
0.9915625
0.9890625
0.99125
train_loss : 200.3954634058682
acc = 0.9910920262336731
count = 125371

epoch : 2


  0%|          | 0/7813 [00:00<?, ?it/s]

...

  0%|          | 0/7813 [00:00<?, ?it/s]

0.9896875
0.99125
0.9940625
0.988125
0.9928125
0.9925
0.9878125
0.993125
0.9915625
0.990625
0.991875
0.993125
0.9890625
0.9871875
0.9925
0.990625
0.991875
0.9915625
0.9878125
0.9925
0.99125
0.9915625
0.9925
0.9928125
0.99375
0.9921875
0.99
0.9915625
0.989375
0.9884375
0.99125
0.993125
0.993125
0.99
0.9890625
0.9925
0.99
0.9921875
0.99375
0.9928125
0.993125
0.989375
0.9928125
0.993125
0.99375
0.99125
0.993125
0.9915625
0.991875
0.993125
0.99125
0.9928125
0.9925
0.9884375
0.991875
0.9871875
0.9915625
0.9921875
0.99125
0.9921875
0.9928125
0.9946875
0.990625
0.99125
0.9928125
0.9915625
0.9915625
0.99
0.990625
0.991875
0.991875
0.99375
0.9921875
0.99375
0.9928125
0.991875
0.991875
0.9896875
train_loss : 189.17535426216637
acc = 0.9915480017662048
count = 125631

epoch : 5


  0%|          | 0/7813 [00:00<?, ?it/s]

0.9909375
0.9909375
0.9903125
0.990625
0.9878125
0.99125
0.9903125
0.9915625
0.99125
0.9878125
0.99375


KeyboardInterrupt: 

In [3]:
class testDataset(Dataset):
    def __init__(self, data, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.code1 = data['code1'].to_numpy()
        self.code2 = data['code2'].to_numpy()

    def __len__(self):
        return len(self.code1)

    def __getitem__(self, idx):
        code1 = self.code1[idx]
        code2 = self.code2[idx]
        
        encoding = self.tokenizer(
            code1,
            code2,
            #add_special_tokens=True,
            max_length=self.max_len,
            truncation=True,
            return_tensors="pt",
            padding='max_length'
        )
        

        return {'input_ids': encoding['input_ids'][0],
                'attention_mask': encoding['attention_mask'][0]}


In [4]:
test_data = pd.read_csv('./test_refine_new_test.csv')
#model_name = 'microsoft/graphcodebert-base'
model_name = 'codesage/codesage-small'

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.truncation_side = 'left'


test_dataset = testDataset(test_data, tokenizer, max_len=1024)

test_loader = DataLoader(test_dataset, batch_size=75, shuffle=False, num_workers=4)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
model = AutoModelForSequenceClassification.from_pretrained("codesage-small-left-test-freeze/cpp-3", trust_remote_code=True)
model.to(device)
model.eval()
with torch.no_grad():
    test_preds = []
    for i, batch in tqdm(enumerate(test_loader),total=len(test_loader)):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, axis=1)
        test_preds.extend(preds.cpu().numpy())
submission = pd.DataFrame({'pair_id': test_data['pair_id'], 'similar': test_preds})
submission.to_csv('./submission.csv', index=False)

  0%|          | 0/7934 [00:00<?, ?it/s]

In [6]:
sum(pd.read_csv('submission.csv')['similar'])

292041